In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Exploračná analýza

## Tabuľka events

In [2]:
events = pd.read_csv('vi_data/events_train.csv')
events.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
10573559,9c321e1a-35fe-40d0-89bb-4352399bb12a,2019-08-16 14:10:44.785572 UTC,view_item,4fe102d6-647d-4a54-b564-b8240267930e,Lee Cooper Check Dress Ladies,Women>Women's Clothing>Dresses>Summer and beac...,25.13
4949068,85c73c85-2674-47f9-973e-0ad78699d5fc,2019-07-24 21:22:02.874581 UTC,view_item,871f84fc-be1d-4094-bc01-2fc311bc7410,Bafiz Team Mens Mule Slippers,Men>Men Footwear>Sandals & Flip-Flops>Sliders,13.32
7569559,16115860-32d0-43e5-94d8-7cfdba0dfe21,2019-08-05 02:59:18.795894 UTC,view_item,ac3e0965-6b1d-4538-a0ff-b71ffdb38808,Lee Cooper Classic T Shirt Ladies,Women>Women's Clothing>T-Shirts>Short Sleeve,12.41
11598572,7056cac1-3a35-48f9-8d42-50e5f7c6a9d9,2019-08-20 19:27:47.981375 UTC,view_item,214001e6-c3da-42de-96ee-fb01e3aead22,Slazenger Three Quarter Track Pants Junior Boys,Children>Boy's Clothing>Shorts>Casual shorts,9.88
9859887,f407c03d-f0f0-41c0-8adb-5339db0a773e,2019-08-13 17:40:18.835259 UTC,view_item,e6502e92-7c3c-435f-93ed-d8a52acd6c88,Women's Sports dress Numoco 139,Women>Women's Clothing>Dresses>Casual Dress,19.24
4592148,9557b3ae-710b-4e7b-a7a3-23da9ef821eb,2019-07-23 07:30:21.870528 UTC,view_item,15e4e32c-41b7-4a8d-87e6-81f16d22c599,Slazenger Mens Swimming Trunks,Sports>Water Sports>Men's Shorts and Swimwear,11.49
12934085,d382e83e-6785-4f2b-ac9b-33c787c9f341,2019-08-26 04:24:03.224646 UTC,view_item,096eb8cd-ffd8-4225-8de1-79bafe6796c0,Ombre Clothing Men's hoodie B821,Men>Men Clothing>Sweatshirts>Hoodies,13.76
6669225,47a6fcd1-3750-458b-9db6-c42c5df18f2c,2019-08-01 14:25:53.15073 UTC,view_item,f43e7f41-691f-4474-939c-a7487d82ce3c,Crafted Junior Boys 3 Pack Jog Set,Children>Boy's Clothing>Boy's sets,19.87
1923860,f94759ee-4561-45b9-9947-e4421a0f66af,2019-07-10 09:49:53.373104 UTC,view_item,8b9db101-d64e-434c-bc1e-0bf1e533907e,Nike Air Bella 2 Trainers Ladies,NaN,93.06
8056460,1a457088-4456-4dbe-9b64-a7e053bdf209,2019-08-06 18:46:23.058316 UTC,view_item,7d1c1e91-88bb-4add-9df3-bdfa696d4a15,Nike Air Max LTD 3 Mens Trainers,Men>Men Footwear>Sneakers>Low sneakers,126.40


In [3]:
events.shape

(14614385, 7)

In [4]:
events.event_type.unique()

array(['view_item'], dtype=object)

## Tabuľka purchases

In [5]:
purchases = pd.read_csv('vi_data/purchases_train.csv')
purchases.sample(10) 

,customer_id,timestamp,event_type,product_id,title,category_name,price
39111,87d5c1e3-9c9c-42cc-813d-159b207a0f6d,2019-07-15 06:55:09.312518 UTC,purchase_item,3868ea1a-8dff-4078-84ed-6b03c71f7923,Lee Cooper Fashion T Shirt Ladies,NaN,NaN
126891,424e041e-703f-4e8e-940b-157c6b7242a4,2019-08-12 18:49:57.774208 UTC,purchase_item,42db8156-68db-47db-b441-4c49aa08e28e,Lee Cooper Men's boxers 5-pack,NEW IN,12.08
94377,95dc6965-1dac-494f-8865-2fc09a91997f,2019-08-03 17:51:15.988554 UTC,purchase_item,717e1bc2-ad01-4b50-a44e-860f6ac530eb,Men's shorts 4F H4L19-SKMT001A,Men>Men Clothing>Shorts>Sports shorts,10.25
119944,248f435a-a05b-4981-ace0-48a34fe50a9b,2019-08-10 18:56:43.496038 UTC,purchase_item,8369aebc-fba5-4957-b3fb-7da05f327dff,Lee Cooper Full Zip Hoody Mens,Men>Men Clothing>Sweatshirts,16.68
13448,84ddda30-c5bb-47bf-ac8d-5c10aa107714,2019-07-06 04:20:42.79311 UTC,purchase_item,a39ecbca-53ba-41e6-96db-90772370fdf8,Lee Cooper Classic T Shirt Ladies,NaN,NaN
67023,8c20882c-75ed-4709-9e27-efecaacf5ab5,2019-07-26 00:46:46.402741 UTC,purchase_item,f9c39709-e8c7-4789-8197-6556c8f5f2c6,Ombre Clothing Men's socks U39 - mix 4,Men>Men Clothing>Underwear>Socks,4.41
96544,9385ef99-39d5-4694-9862-4760ec8fc4be,2019-08-04 10:09:47.793583 UTC,purchase_item,3d99f0f7-c341-4ec1-ac6c-6e4a1ee38cdc,Slazenger Woven Shorts Mens,Men>Men Clothing>Shorts>Sports shorts,14.72
89997,f68293aa-41ee-464c-9ea7-f18fb084d693,2019-08-02 12:50:22.474882 UTC,purchase_item,1fdd9bbd-6990-4cb6-ac7b-d98afa6540f1,Lee Cooper Slip On Zip Ladies Trainers,Women>Women's Footwear>Slip-on,22.06
111670,9119501b-b509-4e9d-ae2e-bfd3c8454cdc,2019-08-08 10:07:28.816279 UTC,purchase_item,27ebd864-91c0-4694-bcc3-dd71c94be92a,Men's T-shirt Lee Cooper 1P,Men>Men Clothing>T-Shirts & Polo shirts>Short ...,4.79
55876,10dc18a9-545f-44fd-b188-be4049938ef7,2019-07-21 11:33:22.733798 UTC,purchase_item,a766ed37-95ad-4fcd-b36d-e8c163b6d304,Official Rolling Stones T Shirt,Men>Men Clothing>T-Shirts & Polo shirts>Short ...,22.74


In [6]:
purchases.shape

(188712, 7)

In [7]:
purchases.event_type.unique()

array(['purchase_item'], dtype=object)

tabulky events a purchaches majú roznaké stĺpce 

## Spojenie events and purchases

In [11]:
data = pd.concat([events, purchases], ignore_index=True)
data.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
11478030,82bef739-7edd-411e-bd89-09258c4e3bc3,2019-08-20 11:15:45.087056 UTC,view_item,554f6c0a-ae21-4602-90c4-7733bf171e20,LA Gear V Neck T Shirt Ladies,Women>Women's Clothing>T-Shirts>Short Sleeve,10.38
7413813,6c5a939e-17a8-43ba-b8b1-4f9000a2c228,2019-08-04 13:39:06.20768 UTC,view_item,765cc8cc-4110-4f94-980d-9856c8d52087,Trendyol Bronze Volan detailed patterned eveni...,Dresses>Women>Women's Clothing>Evening dress,72.23
13124266,1a0f73ea-cc80-46c2-8bbb-ff41a2d3a17c,2019-08-26 17:36:52.216156 UTC,view_item,227e41e2-b621-4ce5-ba4e-0290000b9cc8,Women's lightweight jacket TRIMM CANDY,Women>Women's Clothing>Jackets & Coats>Quilted...,32.29
2184478,cbbce6ed-c244-4401-ab24-d40c587b6d0a,2019-07-11 15:47:35.405104 UTC,view_item,dd5a3959-0460-4819-8708-ad6f1b8f9df7,Puma Retaliate Mens Trainers,NaN,61.10
3153787,5a64027b-88aa-4d8c-b154-51ce770137cc,2019-07-15 21:27:02.828522 UTC,view_item,b3db184c-5bde-4ddc-8778-4e4f927a5ab8,Trendyol Mac high waist Bikini bottom,NaN,12.36
11672271,190a01ce-b735-4c48-8bda-15d390e849d8,2019-08-21 06:35:10.555938 UTC,view_item,d6cd4a45-787d-4002-bbc5-65b80c825e1d,Adidas Shiny Full Zip Tracksuit Boys,Children>Girl's Clothing>Girl's sets,53.83
5744922,425f8019-a607-48fd-880e-79d6b7904125,2019-07-28 19:01:31.188815 UTC,view_item,2b6ce6a0-8ebd-49a0-94ef-d1f3ef8fd861,Made Of Emotion Woman's Dress M276,Women>Women's Clothing>Dresses>Midi dress,36.21
6823129,82c2072e-497d-4993-b00a-d77a52c52c0c,2019-08-02 07:33:39.829517 UTC,view_item,c838021b-4d15-47fc-a603-eac7a1f1b0bd,Nike Arrowz Mens Trainers,Men>Men Footwear>Sneakers>Sport sneakers,74.73
14683153,60d0231b-3ca3-4221-af4c-d471d111daea,2019-07-26 16:09:08.787849 UTC,purchase_item,4ce05861-4306-4b12-829a-9c1ed4561800,Pierre Cardin Small Check Short Sleeve Shirt Mens,Men>Men Clothing>Shirts>short sleeves,14.15
11942473,765809d7-42cf-457e-b3ea-e5dbedb8a701,2019-08-22 07:58:04.64111 UTC,view_item,6f193c8a-79ab-4f30-a853-e9d3ef5f659b,Larica Women’s bow oxfords LR308 grey,Women>Women's Footwear>Oxford shoes & Moccasin...,13.87


In [15]:
rows_num, cols_num = data.shape
print(rows_num)
print(cols_num)

14803097
7


In [16]:
del events
del purchases

## Atribúty

### customer_id

In [11]:
data.customer_id.describe()

count                                 14803097
unique                                 2211386
top       53f841c7-970b-4e0b-96bf-97a95ecfb734
freq                                      4804
Name: customer_id, dtype: object

In [12]:
len(data[data.customer_id.isna()])

0

In [13]:
rows_num/len(data.customer_id.unique())

6.694035776657716

In [14]:
data.groupby(['customer_id']).count()['timestamp'].median()

2.0

In [14]:
customer_rows = data['customer_id'].value_counts()
customer_rows

53f841c7-970b-4e0b-96bf-97a95ecfb734    4804
9eb76d09-8006-4262-b5eb-459556f338a9    3577
546a1146-64be-4549-a472-a8a4a8ca41b0    3189
a3fe2d55-b8fc-4aaa-8f7f-99f6498b191f    3060
68b2c520-5366-4143-a253-04fd96aeb251    2855
                                        ... 
28f3de37-5d91-451b-9d71-c016ff28765a       1
ddaea6ea-cf1e-4ef3-84d0-4d97f5019dab       1
ee2c4b69-2194-4785-a817-76a30ea8a54b       1
db9f09e8-ea9e-4444-986a-39c973a7dfe3       1
3fe6ea93-4d29-4d6f-990a-6a873f19c2ed       1
Name: customer_id, Length: 2211386, dtype: int64

In [28]:
for i, num in enumerate(customer_rows.value_counts()[:3]):
    print(i+1, num)

1 913643
2 329587
3 199175


In [29]:
(913643)/2211386

0.4131540129131685

In [20]:
(913643+329587)/2211386

0.5621949311427313

In [26]:
for i, num in enumerate(customer_rows.value_counts()[3:50:4]):
    print(4*i+4, num)

4 129721
8 45301
12 22832
16 13311
20 8728
24 6193
28 4481
32 3349
36 2677
40 2124
44 1784
48 1405


- 2,211,386 unikátnych zákazníkov
- žiadne NaN hodnoty
- v priemere vyše 6 záznamov na zákazníka
    - ale median je iba 2
    - 913,643 (41%) zákazníkov má len 1 záznam
    - 56% zákazníkov má 2 záznamy

### timestamp

In [16]:
data.timestamp.describe()

count                           14803097
unique                          14802889
top       2019-07-22 04:33:52.889148 UTC
freq                                   4
Name: timestamp, dtype: object

In [17]:
data.loc[data['timestamp'] == "2019-07-22 04:33:52.889148 UTC"]

,customer_id,timestamp,event_type,product_id,title,category_name,price
4361630,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,5bbe88b4-9650-4c1a-8f95-9a09c264b2c9,Hot Tuna Crew T Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts,12.84
4361631,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,248faf65-d1e2-4d7b-84c8-ce1c459ee086,Lonsdale Jersey Polo Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts>Polo S...,12.90
4361632,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,377bdb75-6f46-4c29-b559-9af0c6ba2ae8,Pierre Cardin Web Belt Shorts Mens,Men>Men Clothing>Shorts,17.12
4361633,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,b2bcef2f-df2d-4c23-aee7-73ca112ef815,Lee Cooper Men's boxers 5-pack,Men>Men Clothing>Underwear>Boxer shorts & trunks,13.10


In [47]:
# data["timestamp"] = pd.to_datetime(data["timestamp"])

In [18]:
min(data.timestamp)

'2019-07-01 00:00:01.101977 UTC'

In [19]:
max(data.timestamp)

'2019-08-31 23:59:59.536067 UTC'

In [7]:
len(data[data.timestamp.isna()])

0

- dátumy z intervalu 2 mesiacov
- žiadne NaN hodnoty
- existujú záznamy typu `view_item`, kde je daný zákazník v danom čase a rôzne produkty. 

### event_type

In [21]:
data.event_type.unique()

array(['view_item', 'purchase_item'], dtype=object)

In [22]:
len(data[data.event_type.isna()])

0

In [23]:
len(data.loc[data['event_type'] == "view_item"])/rows_num

0.9872518568242848

In [24]:
len(data.loc[data['event_type'] == "purchase_item"])/rows_num

0.01274814317571519

- dva typy: `view_item` a `purchase_item`
- pomer je 98.7% ku 1.27%
- žiadne NaN hodnoty

### product_id

In [25]:
data.product_id.describe()

count                                 14803097
unique                                  123837
top       31ae211d-57a9-4bfa-b95f-40776af03c3f
freq                                     52736
Name: product_id, dtype: object

In [60]:
len(data[data.product_id.isna()])

0

In [61]:
rows_num/len(data.product_id.unique())

119.53694776197744

In [62]:
data.groupby(['product_id']).count()['timestamp'].median() 

17.0

In [32]:
product_rows = data['product_id'].value_counts()
product_rows

31ae211d-57a9-4bfa-b95f-40776af03c3f    52736
0f943312-7141-4606-abfa-81fd63a5498f    49141
a8f0292d-5fca-42b9-b0d3-b38e7efa416b    48343
e33cea18-ee3f-4e37-83a7-1d88c54d83b8    45421
608f32cd-210c-4af5-99d8-6182a9678cf5    39801
                                        ...  
6b0e71d0-cdca-4160-8df1-ae7b43ef967c        1
630d92d1-ed2b-4a75-a778-3dacba195eb5        1
0633f706-85d4-4ac6-b34d-7f653fa7139d        1
a3c46419-d8fb-4273-88ba-06dc92b42696        1
326e712f-9733-4761-b458-217f3f695833        1
Name: product_id, Length: 123837, dtype: int64

In [36]:
for i, num in enumerate(product_rows.value_counts()[:50:4]):
    print(4*i+1, num)

1 15982
5 3582
9 2365
13 1792
17 1376
21 1139
25 948
29 810
33 694
37 624
41 528
45 501
49 436


- 123,837 unikátnych produktov
- žiadne NaN hodnoty
- v priemere vyše 119 záznamov na produkt
    - maximum je 52,736
    - minimum je 1
    - median je 17
    - 15,982 produktov má len 1 záznam

### title

In [63]:
data.title.describe()

count                     14801284
unique                       71440
top       Lee Cooper Marl Backpack
freq                        168766
Name: title, dtype: object

In [72]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].title.unique()

array(['Batoh Lee Cooper Marl', 'Lee Cooper Marl Backpack'], dtype=object)

In [64]:
len(data[data.product_id.isna()])

0

- 71,440 unikátnych označení (našli sme 123,837 unikátnych produktov)
    - produkt môže mať viac rôznych označení
- žiadne NaN hodnoty

### price

In [85]:
data.price.describe().apply(lambda x: format(x, 'f'))

count    14758211.000000
mean           33.122065
std           121.532833
min             0.060000
25%            12.710000
50%            21.860000
75%            39.490000
max         32636.460000
Name: price, dtype: object

In [87]:
np.nanmedian(data.price)

21.86

In [73]:
len(data[data.price.isna()])

44886

In [89]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].price.unique()

array([12.9 , 12.73, 13.11, 13.32, 12.49, 12.71, 13.45, 12.65, 12.83,
       11.46, 11.8 , 11.44, 11.38, 12.1 , 11.24, 11.61, 11.99, 11.55,
        9.55,  9.56, 10.61, 10.63, 13.49,  7.32, 12.14, 12.27, 12.33,
       13.04, 12.92, 12.51, 12.11, 12.45, 12.35, 13.09, 16.84, 16.41,
       16.11, 16.03, 16.23, 15.95, 16.  , 15.58, 16.39, 16.97, 13.65,
       14.75, 13.98, 14.02, 14.05, 14.22, 14.38, 14.87, 14.12, 14.37,
       12.46, 12.37, 13.12, 12.04, 13.02, 12.39, 12.68, 12.55, 12.69,
         nan])

- priemer ceny je cca 33
- median ceny je cca 22
- 44,886 NaN hodnôt
- jeden produkt môže mať viac cien

### category_name

In [90]:
data.category_name.describe()

count                                   11367365
unique                                       713
top       Men>Men Footwear>Sneakers>Low sneakers
freq                                      462179
Name: category_name, dtype: object

In [103]:
for pid in data.product_id.sample(10):
    unique_categories = data.loc[data['product_id'] == pid].category_name.unique()
    print(len(unique_categories), unique_categories)

2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
3 [nan "Women>Women's Clothing>Shirts"
 "Women>Women's Clothing>Shirts>Patterned shirts"]
2 [nan "Women>Women's Clothing>Jackets & Coats>Coats"]
2 [nan "Women>Women's Footwear>Slip-on"]
2 [nan "Women>Women's Clothing>Dresses>Wedding dress"]
2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
2 [nan "Sports>Running>Running Shoes>Women's Running Shoes"]
2 [nan 'Accessories >Backpacks and Bags>Fashion backpacks']
3 ["Women>Women's Clothing>Pants & Jeans>Pants"
 "Women>Women's Clothing>Pants >Pants"
 "Women>Women's Clothing>Pants >Single-colored pants"]
2 [nan "Women>Women's Clothing>Jumpsuits"]


In [96]:
category_name_nans_num = len(data[data.category_name.isna()])
category_name_nans_num

3435732

In [95]:
category_name_nans_num/rows_num

0.23209548650529008

In [45]:
data["category_name"].str.split(">", expand = True).shape

(14803097, 7)

In [40]:
data_15K = data.sample(15000)

In [116]:
categories = data_15K["category_name"].str.split(">", expand = True)

Keďže `data` má skoro 15M záznamov, spracovanie v niektorkých prípadoch trvá veľmi dlho. Vybrali sme len 15K.

In [123]:
categories.sample(5)

,0,1,2,3,4
2563389,NaN,NaN,NaN,NaN,NaN
7189282,Women,Women's Footwear,Balerinas,None,None
11478593,Men,Men Clothing,T-Shirts & Polo shirts,Short Sleeve,None
6003198,Women,Women's Clothing,Swimmsuits & bikins,One-piece,None
1495798,NaN,None,None,None,None


In [136]:
for i in range(5):
    print('col_{} unique values ='.format(i), len(categories[i].unique()))
    print('col_{} NaN values ='.format(i), len(categories[categories[i].isna()]))
    print()

col_0 unique values = 21
col_0 NaN values = 3455

col_1 unique values = 38
col_1 NaN values = 3551

col_2 unique values = 154
col_2 NaN values = 3669

col_3 unique values = 216
col_3 NaN values = 6845

col_4 unique values = 49
col_4 NaN values = 14513



In [126]:
categories[0].unique()

array([nan, 'Men', 'Women', 'Sports', "Women's Clothing", 'Children',
       'Accessories ', 'Backpacks and Bags', 'Men Clothing',
       "Boy's Clothing", 'Sale - Men', 'Sale - Women', 'Dresses',
       'NEW IN', 'Unlisted', "Children's Footwear", 'strom kategorii',
       'T-Shirts', 'NY FASHION WEEK', 'Sale - Children',
       'Delivery Immediately'], dtype=object)

- **iba** 713 unikátnych kategórií (našli sme 123,837 unikátnych produktov)
- jeden produkt môže mať rôzne `category_name` hodnoty, vrátane NaN
- 23.2% záznamov je NaN

- po rozparsovaní stromu `category_name` vzniklo 7 stĺpcov (t.j. výška stromu)
- smerom od koreňa rastie počet NaN hodnôt a unikátnych hodnôt
- vyskytujú sa tématicky rovnaké len inak napísané kategórie


### Diskusia
- málo záznamov od jedného zákazníka

Potreba zjednotiť hodnoty:
- `title`
- `category_name`
- možno `cena`?

---

# Predspracovanie a oprava dát

In [17]:
categories = data["category_name"].str.split(">", expand = True, n=3)
del categories[3]
del data["category_name"]

In [20]:
categories = categories.rename(columns={0: "cat_0", 1: "cat_1", 2: "cat_2"})

In [28]:
data = pd.concat([data, categories], axis=1, sort=False)

Vypasovali sme prve tri kategórie zo stromu kategorií a pridali ako samostatné stĺpce

In [31]:
product_ids = list(data.product_id.unique())

In [62]:
# flatten = lambda l: set(item for sublist in l for item in sublist)
# products = {}
# for pid in product_ids:
#     try:
#         atr = {}
#         rows = data[data.product_id == pid]
#         atr['title'] = flatten([i.split() for i in list(rows.title.unique())])
#         atr['price'] = np.median(rows.price)
#         atr['cat_0'] = list(rows.cat_0.dropna().unique())[0]
#         atr['cat_1'] = list(rows.cat_1.dropna().unique())[0]
#         atr['cat_2'] = list(rows.cat_2.dropna().unique())[0]
#     except IndexError:
#         pass
#     products[pid] = atr

In [58]:
len(products)

1

In [33]:
data[data.product_id == product_ids[0]]

,customer_id,timestamp,event_type,product_id,title,price,cat_0,cat_1,cat_2
0,204685f9-00f0-4a36-bab1-e0b74673c3b1,2019-07-01 00:00:01.101977 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,747.49,NaN,None,None
50497,4246f982-6ec4-408e-ac13-0b1ae1633750,2019-07-01 09:14:26.818799 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,747.49,NaN,None,None
56918,20c0c866-ef84-4308-809f-2fa0104aaa76,2019-07-01 09:47:45.62947 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,747.49,NaN,None,None
120723,26cccf04-9acb-4231-b72b-45186dabfb6e,2019-07-01 15:06:00.006045 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,747.49,NaN,None,None
211849,4262beec-3a73-45ca-a6ee-d4d5587f8777,2019-07-01 21:38:43.882073 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,747.49,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5328958,1df90fc5-6fd3-492a-ac98-323f81df6890,2019-07-26 20:38:42.181133 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,695.16,Men,Men Clothing,Jackets & Coats
5556108,16c1770b-e3cd-4f22-b6a0-4b303ba736df,2019-07-28 05:24:05.449254 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,695.16,Men,Men Clothing,Jackets & Coats
9860697,141b70c8-fa1b-47b4-8f99-68b416af2dbc,2019-08-13 17:43:44.032062 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,771.31,Men,Men Clothing,Jackets & Coats
9860738,141b70c8-fa1b-47b4-8f99-68b416af2dbc,2019-08-13 17:43:56.556062 UTC,view_item,f130c542-d793-41d0-88f6-d1c52b4fe8f0,Michael Kors Nappa Leather Racer Jacket,771.31,Men,Men Clothing,Jackets & Coats


In [ ]:
# data.groupby('customer')['...'].apply(list)

In [27]:
# categories_onehot = pd.get_dummies(categories)
# categories_onehot.sample(5)

# Rozdelenie dát do vzoriek

## Odstránenie používateľov s jedným záznamom

In [38]:
rows_num

14803097

In [39]:
data.customer_id.duplicated()

0         False
1         False
2         False
3          True
4         False
          ...  
188707     True
188708     True
188709     True
188710     True
188711     True
Name: customer_id, Length: 14803097, dtype: bool

## 

In [12]:
train_ratio, validation_ratio, test_ratio = 0.8, 0.20

In [9]:
len(data)

14803097

In [ ]:
del data

# Primitívny odporúčač

Odporúčanie najpredávanejších produktov

In [69]:
class PrimitiveRecomender:
    def __init__(self, df):
        self.tops = list(df['product_id'].value_counts().head(10).index)
    def recomend(self, id):
        # ID is not used in any way
        return self.tops

# Obsahový odporúčač

In [130]:
from sklearn.cluster import KMeans

In [131]:
X = np.array([[1, 2], [1, 4], [1, 0],
            [10, 2], [10, 4], [10, 0]])

In [132]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

In [133]:
kmeans.labels_

array([1, 1, 1, 0, 0, 0], dtype=int32)

In [134]:
kmeans.predict([[0, 0], [12, 3]])

array([1, 0], dtype=int32)

In [135]:
kmeans.cluster_centers_

array([[10.,  2.],
       [ 1.,  2.]])

In [27]:
class ContentRecomender:
    def __init__(self, train):
        self.train = train
    def recomend(self, id):
        return [1, 2, 3, 4, 5]

In [32]:
def recomendation(recomender, data):
    results = {}
    for id in data:
        results[id] = recomender.recomend(id)
    return results

# Evaluácia

## Metriky

Použité implementácie z [Ranking Metrics](https://gist.github.com/bwhite/3726239)

In [101]:
def precision_at_k(r, k):
    """Score is precision @ k
    Relevance is binary (nonzero is relevant).
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

In [98]:
def relevance(predicted, test):
    r = []
    for x in predicted:
        r.append(1 if x in test else 0)
    return r

## Výsledky

## 1

In [70]:
primitive_recomender = PrimitiveRecomender(data)

In [95]:
predicted_products = primitive_recomender.recomend(0)

In [105]:
test_customers = list(data.customer_id.unique()[:100])

In [106]:
test_products = {}
for cid in test_customers:  
    test_products[cid] = list(data[data.customer_id == cid]['product_id'])

In [114]:
np.mean([precision_at_k(relevance(test_products[key], predicted_products), 3) for key in test_customers])

ValueError: Relevance score length < k

## 2

In [35]:
test_customers = [0, 1, 2]
test_products = {0: [1, 2, 5], 1: [3, 4, 5], 2: [1, 2, 3, 4]}

In [33]:
content_recomender = ContentRecomender(None)

In [36]:
predicted_products = recomendation(content_recomender, test_customers)

In [37]:
np.mean([precision_at_k(relevance(test_products[key], predicted_products[key]), 3) for key in test_customers])

NameError: name 'precision_at_k' is not defined

## 3

# Poznámky

1. koľko odporúčaní generovať?
2. používalov s 1 záznamom vyhodiť?